#  INVERTED INDEXES LOADING


In [7]:
from termcolor import colored
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm

wremove = ['-','â€','â€¦','”']
filedata = []
def remove_prepositions(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    filtered = [word for word, pos in tagged if pos not in ['IN', 'TO']]
    return ' '.join(filtered)
# CREATING INSTANCES OF STEMMING AND LEMMITIZATION
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
stopwords = ["was","wasn't","was not","donot","don't","there","here","never","n't","s","been","will","would","wouldn't","hasn't","has not","haven't","have not","doesn't","had","hadn't","had not","upto","and","or","not","nt","that","this","it","i","why","who","how","what","when","he","she","their","you","but","because","then","which","him","her","their","where","could","an"]
translator = str.maketrans('', '', string.punctuation.replace('-', ''))
# READING STOP WORDS FROM FILE
with open("C:\\Users\\DELL\\Desktop\\boolean\\Stopword-List.txt", "r") as f:
    contents = f.read()
    words = contents.split()

    # Store the words in an array
    for word in words:
        stopwords.append(word)

documents = []
wholecontent = ""
# READING ALL 30 FILES AND APPLY PREPROCESSING 
for x in range(1, 31):
    t = "C:\\Users\\DELL\\Desktop\\boolean\\{}.txt".format(x)
    with open(t, "r") as f:
        contents = f.read()
        # FOR REMOVING LINKS AND SPECIAL CHARACTRS FROM FILE
        contents = re.sub(r"&nbsp;.*\n", " ", contents)
        contents = re.sub(r"â€¢.*\n", " ", contents)
        # STEMMING PROCESS
        contents = nltk.word_tokenize(contents)
        contents = [stemmer.stem(word) for word in contents]
        contents = ' '.join(contents)
        # LEMMITIZATION PROCESS
        contents = " ".join([lemmatizer.lemmatize(word, pos='v') if pos.startswith('V') else lemmatizer.lemmatize(word) for word, pos in nltk.pos_tag(nltk.word_tokenize(contents))])
        # REMOVING PUNCTUATION MARKS
        contents = contents.translate(translator)
        # APPLYING CASE FOLDING
        contents = contents.lower()
        # REMOVING PREPOSITIONS
        contents = remove_prepositions(contents)
        contents = ' '.join(word for word in contents.split() if word not in stopwords)
        # GATHERING ALL PREPROCESSED DATA
        filedata.append(contents)
        wholecontent += contents
        contents = ""
# BUILDING INDEXES
wholecontent = wholecontent.split()
wholecontent = set(wholecontent)
wholecontent = sorted(list(wholecontent))
wholecontent = [element for element in wholecontent if element not in wremove]

inverted_index = {}
# LOADING BAR PARALLEL WITH INDEX BUILDING
for term in tqdm(wholecontent, leave=False):
    inverted_index[term] = []
    for x in range(1, 31):
        data = filedata[x-1].split()
        if term in data:
            inverted_index[term].append(x)





# YOUR DICTIONARY

In [8]:
for key, value in inverted_index.items():
    print(colored(key, "red", attrs=["bold", "underline"]), ":", colored(f"[{value}]", "green", attrs=["bold"]), "\n")

0 : [[9, 16, 23]] 

04-feb-2023 : [[23]] 

1 : [[1, 6, 10, 16, 17, 27]] 

1-0 : [[22]] 

1-1 : [[30]] 

1-13 : [[30]] 

1-16 : [[4]] 

1-4 : [[30]] 

10 : [[9, 18, 22, 24]] 

10-over : [[28]] 

100 : [[12, 26]] 

100-run : [[23]] 

1002nd : [[7]] 

101 : [[30]] 

103 : [[15, 27]] 

10th : [[9, 18]] 

11 : [[7, 13, 17, 28, 30]] 

111 : [[2]] 

112 : [[16, 23]] 

113 : [[16]] 

115 : [[15]] 

116 : [[23]] 

118 : [[28]] 

11am : [[5]] 

11th : [[15, 22, 30]] 

12 : [[1, 2, 7, 18, 22, 23, 25, 30]] 

120 : [[19]] 

122 : [[18]] 

1225 : [[22]] 

125 : [[27]] 

126 : [[27]] 

127 : [[2]] 

12th : [[12]] 

13 : [[1, 9, 14, 17, 27, 30]] 

13-0-110-2 : [[7]] 

130 : [[1, 2]] 

131 : [[28]] 

133 : [[7]] 

137 : [[2]] 

138 : [[7]] 

13th : [[18, 27]] 

14 : [[2, 10, 18, 28]] 

140 : [[17]] 

144 : [[19]] 

146kph90mph : [[28]] 

149 : [[18]] 

14th : [[4, 27]] 

15 : [[2, 4, 7, 8, 22, 27]] 

15-year-old : [[1]] 

150 : [[15, 18]] 

151 : [[18]] 

158 : [[7]] 

15th : [[12]] 

16 : [[1, 2, 14, 

# APPLY BOOLEAN QUERIES

In [9]:
import re
import ipywidgets as widgets
from IPython.display import display
# function for AND QUERIES
def intersection(postings1, postings2):
    i = j = 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] == postings2[j]:
            result.append(postings1[i])
            i += 1
            j += 1
        elif postings1[i] < postings2[j]:
            i += 1
        else:
            j += 1
    return result
#FUNCTION FOR OR QUERIES
def union(postings1, postings2):
    i = j = 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] < postings2[j]:
            result.append(postings1[i])
            i += 1
        elif postings2[j] < postings1[i]:
            result.append(postings2[j])
            j += 1
        else:
            result.append(postings1[i])
            i += 1
            j += 1
    
    # Append any remaining elements in postings1 and postings2
    result.extend(postings1[i:])
    result.extend(postings2[j:])
    
    return result
# INTERMEDIATE FUNCTION USED IN BOOLEAN QUERIES FOR AND OR OPERATORS
def generalfun(posting1, posting2, oper):
    if oper == 'AND':
        return intersection(posting1, posting2)
    elif oper == 'OR':
        return union(posting1, posting2)
#FUNCTION FOR NOT OPERATOR
def NOTfun(posting):
    temp = []
    for x in range(1, 31):
        if x not in posting:
            temp.append(x)
    return temp
# Define the regular expression.
query_re = re.compile(r'^((\w+)(\s+(AND|OR|NOT)\s+(\w+))?(\s+(AND|OR|NOT)\s+(\w+))?)?$')

# Define the input widgets.
query_input = widgets.Text(placeholder='Enter a boolean query')

# Define the output widgets.
output_label = widgets.Label()
output_list = widgets.SelectMultiple()

# Define the button widget and its click event handler.
button = widgets.Button(description='Search')

def on_button_click(_):
    query = query_input.value
    # MAKING ARRAY OF QUERY ELEMENTS
    parse = query.split()
    operator = ['AND','NOT','OR']
    # DEALING WITH NULL QUERY.
    if len(parse) == 0:
        output_label.value = 'Error: Blank input'
    #CHECKING IF QUERY MATCHES REGULAR EXPRESSION
    elif query_re.match(query):
        for x in range(0, len(parse)):
            if parse[x] not in operator:
                #APPLYING STEMMING AND LEMMITIZATION TO QUERY TERMS 
                parse[x] = parse[x].lower()
                parse[x] = stemmer.stem(parse[x])
                parse[x] = lemmatizer.lemmatize(parse[x])
        # LENGTH 1 MEANS RETURN POSTING LIST DIRECTLY
        if len(parse) == 1:
            posting = []
            if parse[0] in inverted_index:
                posting = inverted_index[parse[0]]
            output_label.value = 'Results:'
            output_list.options = posting
        # LENGTH 3 MEANS QUERY WITH 2 TERMS
        elif len(parse) == 3:
            posting1 = []
            posting2 = []
        #VALIDATING QUERY TERMS
            if parse[0] in inverted_index:
                posting1 = inverted_index[parse[0]]
            if parse[2] in inverted_index:
                posting2 = inverted_index[parse[2]]
            result = generalfun(posting1, posting2, parse[1])
            output_label.value = 'Results:'
            output_list.options = result
        # LENGTH 5 MEANS QUERY WITH 3 TERMS
        elif len(parse) == 5:
            posting1 = []
            posting2 = []
            posting3 = []
        #VALIDATING QUERY TERMS AND RETURN ANSWER
            if parse[0] in inverted_index:
                posting1 = inverted_index[parse[0]]
            if parse[2] in inverted_index:
                posting2 = inverted_index[parse[2]]
            if parse[4] in inverted_index:
                posting3 = inverted_index[parse[4]]
            result = generalfun(generalfun(posting1, posting2, parse[1]), posting3, parse[3])
            output_label.value = 'Results:'
            output_list.options = result
    #HANDLING NOT QUERY
    elif len(parse) == 2 and parse[0] == 'NOT':
        posting = []
        if parse[1] in inverted_index:
            posting = inverted_index[parse[1]]
        result = NOTfun(posting)
        output_label.value = 'Results:'
        output_list.options = result
    else:
        output_label.value = 'Error: Query is not valid.'

button.on_click(on_button_click)

# Create a container to hold the widgets.
input_box = widgets.VBox([query_input, button])
output_box = widgets.VBox([output_label, output_list])
container = widgets.HBox([input_box, output_box])

# Display the container.
display(container)
